In [1]:
import os
os.chdir('../..')
from pipelines.util import *

In [2]:
# get the data as a dataframe from EDD using DuckDB
URL = 'https://github.com/economic-analytics/edd/raw/main/data/parquet/PROD_LAD.parquet'
con = duckdb.connect()
data = con.execute(f"SELECT * FROM '{URL}'").fetchdf()

In [3]:
# data['unix'] = pd.to_datetime(data['dates.date'], format=f'%Y-%m-%d').astype(int).div(10**6).astype(int)
# data['decimal_date'] = data['unix'].div((86400*365.25)).add(1970).round(2)
data = add_decimal_date_to_dataframe(data, 'dates.date', div=10**6)

In [4]:
data = most_recent_date(data, 'decimal_date')

Getting data for decimal_date == 2022.0


In [5]:
data = data.pivot(index=['geography.code', 'geography.name'], columns='variable.name', values='value')

In [6]:
data.to_csv('src/themes/innovation-change/productivity/_data/most_recent_productivity_local_authority.csv')

In [7]:
time_updated('src/themes/innovation-change/productivity/_data/updated.yaml')

Timestamp added to file src/themes/innovation-change/productivity/_data/updated.yaml


In [8]:
edd_last_updated_next_updated('PROD_LAD')

         id                         desc last_update next_update
0  PROD_LAD  Regional Productivity (LAD)          NA          NA
